In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Feature combiner

Take all the datasets we've created and combine into single vectors for train and test

## Load datasets

In [2]:
dataset_folder = os.path.join(os.pardir, 'Datasets')
feature_files = [
    'FEATURE_entity_counts_{}.csv',
    'magic_feature_{}.csv',
    'FEATURE_topic_sharing_{}.csv',
    'question_type_features_{}.csv',
    'feature_tools_trans_{}.csv',
    'FEATURE_tfidf_{}.csv',
    'graph_features_{}.csv',
    'nlp_features_{}.csv',
    'non_nlp_features_{}.csv',
    'FEATURE_distance_{}.csv',
    'q_features_{}.csv',
    'lstm_feature_{}.csv'
]

In [3]:
X_train = pd.read_csv(os.path.join(dataset_folder, 'train.csv'), index_col='id')
X_test = pd.read_csv(os.path.join(dataset_folder, 'test.csv'), nrows=2345796, index_col='test_id')

train_shape = X_train.shape
test_shape = X_test.shape

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
for feature_set in feature_files:
    train_feature_set = feature_set.format('train')
    test_feature_set = feature_set.format('test')
    
    try:
        X_train_feature = pd.read_csv(os.path.join(dataset_folder, train_feature_set), index_col='id')
        try:
            X_test_feature = pd.read_csv(os.path.join(dataset_folder, test_feature_set), 
                                         index_col='test_id', nrows=2345796)
        except ValueError as e:
            X_test_feature = pd.read_csv(os.path.join(dataset_folder, test_feature_set), 
                                         index_col='id', nrows=2345796)           
        
        assert X_train_feature.index.nunique() == X_train_feature.shape[0], \
            'Index not unique in feature set {}'.format(train_feature_set)
        assert X_test_feature.index.nunique() == X_test_feature.shape[0], \
            'Index not unique in feature set {}'.format(test_feature_set)
        
    except FileNotFoundError as e:
        print('Can\'t add feature {} as train and test sets don\'t both exist.'.format(feature_set))
        continue
        
    X_train = X_train.merge(X_train_feature, left_index=True, right_index=True, how='left')
    X_test = X_test.merge(X_test_feature, left_index=True, right_index=True, how='left')
        
    assert X_train.shape[0] == train_shape[0], 'Rows added'
    assert X_test.shape[0] == test_shape[0], 'Rows added'
    
y_train = X_train.pop('is_duplicate')

Can't add feature FEATURE_entity_counts_{}.csv as train and test sets don't both exist.


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Can't add feature FEATURE_distance_{}.csv as train and test sets don't both exist.


ValueError: Index id invalid

In [ ]:
X_train

In [ ]:
train_metadata = ['qid1', 'qid2', 'question1', 'question2']
X_train.drop(columns=train_metadata, inplace=True)
test_metadata = ['question1', 'question2']
X_test.drop(columns=test_metadata, inplace=True)

In [ ]:
print(X_train.shape)
X_train.head()

In [ ]:
print(X_test.shape)
X_test.head()

In [ ]:
y_train.shape

## Save final feature sets

In [ ]:
X_train.to_csv('x_train.csv')
X_test.to_csv('x_test.csv')

In [ ]:
y_train.to_csv('y_train.csv', index=False)